In [1]:
import program.app_modules as rs
import os
import pandas as pd
import numpy as np

# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
# url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"

filename = url.split('/')[-1]
workingdir = os.path.abspath('')
datadir = "program/data" # Directory where data is stored
filepath_tsvgz = os.path.join(workingdir, datadir, filename)
filepath_tsv = filepath_tsvgz[0:-3]

print("creating dataframe")

usecols = ['customer_id','product_id','star_rating',]

df = pd.read_csv(filepath_tsvgz,\
                delimiter='\t',\
                encoding="utf-8",\
                error_bad_lines=False,\
                compression='gzip',\
                # dtype=dtype,\
                usecols=usecols)
# Clean dataframe
df = df[df['star_rating'].isin([0,1,2,3,4,5])]
df.astype({'star_rating':np.int8})
df.astype({'customer_id':np.int8})
df.astype({'star_rating':np.int8})

print("created successfully")
print(df.head())

creating dataframe


C:\Users\azomm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


created successfully
        customer_id  product_id star_rating
131072     52158092  1577311310           5
131073      2401115  B004S7VSQ6           5
131074     11978442  0547053649           5
131075     35344608  1846703395           5
131076     42078536  1508598592           5


In [8]:
# Clean dataframe
df = df[df['star_rating'].isin([0,1,2,3,4,5])]
df.astype({'star_rating':int})
df.astype({'customer_id':str})
df.astype({'product_id':str})

customer_id  product_id star_rating
131072       52158092  1577311310           5
131073        2401115  B004S7VSQ6           5
131074       11978442  0547053649           5
131075       35344608  1846703395           5
131076       42078536  1508598592           5
131077       42865177  1476728747           5
131078       11954372  1416592377           5
131079       52632594  178208259X           5
131080       28972524  0385337612           5
131081       10436571  088368280X           5
131082       35649354  0698108701           5
131083       17504792  0778802132           4
131084       36635970  1783281936           4
131085       44546216  0307270165           5
131086       14323084  0898862361           5
131087       45383854  1590525132           5
131088       37101625  0073405981           5
131089       50948585  0718031822           5
131090         984490  0977154122           5
131091       15551011  1465412085           1
131092       13652022  1598428985           5
131093       45004785  1620078538           5
131094       22262158  1476724210           5
131095       11526693  0978534425           3
131096       16026175  1570620199           5
131097       15186063  0380794284           5
131098       36557706  1612620566           4
131099       46331729  0064410161           5
131100       11797429  0913074047           5
131101       22600264  1942192002           5
...               ...         ...         ...
10237146     37633243  1456308009           5
10237147     44495483  1937002101           5
10237148     43165422  1601631987           5
10237149     27329629  0977144119           5
10237150     31031313  0803235372           5
10237151     38346756  1461146798           5
10237152     50296069  1605981109           3
10237153     13045047  141854891X           5
10237154     15842225  0071714065           5
10237155     12051324  0062059939           4
10237156     47283178  1118072642           5
10237157     30677149  1936447398           4
10237158     30842160  1451640943           5
10237159     47598274  B0006QMIT4           5
10237160     25691512  1890087025           5
10237161     18494503  1608205584           5
10237162     47710021  0123740371           5
10237163     44660442  059652952X           4
10237164     47777759  0976457490           5
10237165     37328430  1419701789           3
10237166     38937224  1598004697           5
10237167     37060071  1930401825           1
10237168     12731407  0615599877           3
10237169     21950990  0061962767           4
10237170     32858940  1616501642           5
10237171     52004451  B003ZX1EQS           5
10237172     36099005  0786652918           4
10237173     38081951  1608981215           4
10237174     53040283  0670023418           5
10237175     17586627  1475142110           5

[8336587 rows x 3 columns]

In [9]:
from scipy.sparse import csr_matrix, save_npz, load_npz # Sparse matrix
from sklearn.neighbors import NearestNeighbors

# Calculate total number of products and customers (for range calculation)
print("success 1")
prodNo = len(set(df["product_id"])) # Number of products
custNo = len(set(df["customer_id"])) # Number of customers
print("Number of unique products: {}".format(prodNo))
print("Number of unique customers: {}".format(custNo))

# Get unique product and customer IDs and index them
print("success 2")
prodUnique_indexed = dict(zip(np.unique(df["product_id"]), list(range(prodNo))))
custUnique_indexed = dict(zip(np.unique(df["customer_id"]), list(range(custNo))))
prodUnique_reverseIndexed = dict(zip(list(range(prodNo)), np.unique(df["product_id"])))
custUnique_reverseIndexed = dict(zip(list(range(custNo)), np.unique(df["customer_id"])))

# Finally go through each row of dataframe and assign index 
print("success 3")
custIndex = [custUnique_indexed[i] for i in df["customer_id"]]
prodIndex = [prodUnique_indexed[i] for i in df["product_id"]]

# Create indexed matrix
print("success 4")
df_custIndex = pd.DataFrame(custIndex, columns=["custIndex"])
df_prodIndex = pd.DataFrame(prodIndex, columns=["prodIndex"])
df_indexed = pd.DataFrame(df["star_rating"], columns=["star_rating"])
df_indexed.insert(0, "prodIndex", df_prodIndex)
df_indexed.insert(0, "custIndex", df_custIndex)


# Create the sparse matrix based on a matrix with dimensions:
# (no. of unique products x no. of unique customres).
# Description from official scipy documentation:
# csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])
# where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k].

print("...creating sparse matrix...")
df_csr = csr_matrix((df_indexed["star_rating"], (prodIndex, custIndex)), shape=(prodNo, custNo))

msg = "\nFitted the KNN model successfully."

success 1
Number of unique products: 2015314
Number of unique customers: 3910621
success 2
success 3
success 4
...creating sparse matrix...


TypeError: no supported conversion for types: (dtype('O'),)